# PDI

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from os import listdir


In [2]:
#Definindo o filtro de gabor (trocar para log-gabor quando nescessario)
def gabor(lamda, theta, psi, sigma, gamma, M, N):

  (x,y) = np.meshgrid(np.arange(0,N), np.arange(0,M))

  # Rotation 
  xl =  (x-N/2) * np.cos(theta) + (y-M/2) * np.sin(theta)
  yl = -(x-N/2) * np.sin(theta) + (y-M/2) * np.cos(theta)

  gab_filter = np.cos((2*np.pi/lamda)*xl + psi)  
  gab_filter = gab_filter * np.exp((-1/(2*sigma**2))*(xl**2 + gamma**2 * yl**2))

  return gab_filter 

In [3]:
def filter(imgFreq, gb, P, Q, M, N):
    #G = H*F
    Fimg_filtered = gb*imgFreq

    #fft inversa
    img_back = np.fft.ifft2(Fimg_filtered)

    # Multiplicao por (-1)^(x+y)
    mult_img_back = np.zeros([P, Q])
    for x in range(P):
        for y in range(Q):
            mult_factor = (-1)**(x+y)
            mult_img_back[x, y] = img_back[x, y].real * mult_factor

    final_img = mult_img_back[0:M, 0:N]

    return final_img

In [4]:
img_path = 'C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3'

df = pd.DataFrame()

In [5]:
i = 1
classes = []

#para cada imagem:
for img_filename in listdir(img_path):
    print(f'{i}: Processing imag: {img_path}/{img_filename}')
    # Lê imagem e automaticamente converte para 1 canal de cinza
    img = cv2.imread(img_path + '/' + img_filename, cv2.IMREAD_GRAYSCALE)

    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)

    img2 = img.reshape(-1)
    df_image = pd.DataFrame()
    df_image['Original Image'] = img2

    #Padding 
    M, N = img.shape
    img_padded = np.pad(img, ((0, M), (0, N)), 
                        mode='constant', constant_values=0)
    P, Q = img_padded.shape


    # Multiplicao por (-1)^(x+y)
    mult_img = np.zeros([P, Q])

    #for x in range(P):
    #    for y in range(Q):
    #        mult_factor = (-1)**(x+y)
    #        mult_img[x, y] = img_padded[x, y] * mult_factor
    mult_img = img_padded * np.fromfunction(lambda x, y: (-1)**(x+y), img_padded.shape)

    # Converter imagem do domínio do espaço para o domínio da frequência
    f = np.fft.fft2(mult_img)

    # =====================================
    count = 1
    #Cria filtros de gabor
    classes.append(img_filename.split('nr')[0])
    for theta in range(3):
        theta = theta * 60 * (np.pi/180)
        for sigma in (30, 50):
            for lamba  in (25, 50):
                for gamma in (0.5, 1):
                    label = 'Gabor-' + str(count)

                    psi = 0
                    gb = gabor(lamba, theta, psi, sigma, gamma, P, Q)

                    fimg = filter(f, gb, P, Q, M, N)
                    filtered_img = fimg.reshape(-1)
                    df_image[label] = filtered_img
                    count += 1

    count = 0
    flattened = df_image.values.reshape(1, -1)
    df = pd.concat([df, pd.DataFrame(flattened)], ignore_index=True)
    i += 1
df.insert(0, 'classe', classes)
df

1: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3/l1nr001.tif
2: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3/l1nr002.tif
3: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3/l1nr003.tif
4: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3/l1nr004.tif
5: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3/l1nr005.tif
6: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3/l1nr006.tif
7: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3/l1nr007.tif
8: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Swedish Leaf Dataset/leafs 1 to 3/l1nr008.tif
9: Processing imag: C:/Users/mathe/Desktop/Pastas/UFSCar/Perfil 5/PDI/Sw

ValueError: Length of values (675) does not match length of index (225)

# AM

- Note que agora o vetor $df$ armazena as caracteristicas obtidas pelo banco de filtros em cima da imagem

In [6]:
import matplotlib.colors as mcolors
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

In [ ]:
random_seed = 42
df = df.sample(frac=1, random_state=random_seed).reset_index(drop=True)
df



## Decision Tree

In [ ]:
Atributos = df.drop(columns=['classe'])  # Atributos
y = df['classe']  # Variavel objetivo

#Separa testes de treino
X_train, X_test, y_train, y_test = train_test_split(Atributos, y, test_size=0.3, random_state=42)

#Treina modelo
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)

#Usa modelo nos dados de teste
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

## Random Forest

In [ ]:
# Instanciar a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=32, random_state=42)

# Ajustar o modelo para os dados de treinamento
rf_classifier.fit(X_train, y_train)

# Fazer predições nos dados de teste
y_pred = rf_classifier.predict(X_test)

# Evaluate the model (you can use accuracy, precision, recall, etc.)
accuracy = rf_classifier.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

## Naive Bayes

In [ ]:
#Aqui precisa normalizar
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

## SVM

TODO